## AlphaVantage Data Documentation

The following documentation goes through the finance data received from alphaVantage, its output, transformations, normalizing it, and where to stage the data in AWS s3.

Data:

- Income Statement
- Balance Sheet
- Cash Flow
- ETF Profile & Holdings
- Corporate Action


# Income Statement

In [7]:
import polars as pl
import os
from alphaio import AlphaIO
from s3io import S3IO
from datetime import datetime

In [4]:
# load in data for RGLD
alphaio = AlphaIO()
rgld_income = alphaio.get_statement(ticker='RGLD', statement='income')

rgld_income[0]

KeyError: 'ALPHA_VANTAGE_API'

In [3]:
# print the length of the output
print(f"Length of output: {len(rgld_income)}")
print(f"Keys of the first element: {rgld_income[0].keys()}")

Length of output: 1
Keys of the first element: dict_keys(['symbol', 'annualReports', 'quarterlyReports'])


In [4]:
print(f"Length of the annualReports: {len(rgld_income[0]['annualReports'])}")
print(f"Length of the quarterlyReports: {len(rgld_income[0]['quarterlyReports'])}")

Length of the annualReports: 13
Length of the quarterlyReports: 59


In [6]:
print(f"first element of the annualReports: {rgld_income[0]['annualReports'][1]}")
print(f"first element of the quarterlyReports: {rgld_income[0]['quarterlyReports'][1]}")

first element of the annualReports: {'fiscalDateEnding': '2022-12-31', 'reportedCurrency': 'USD', 'grossProfit': '323101000', 'totalRevenue': '603206000', 'costOfRevenue': '133541000', 'costofGoodsAndServicesSold': '94642000', 'operatingIncome': '283709000', 'sellingGeneralAndAdministrative': '34612000', 'researchAndDevelopment': 'None', 'operatingExpenses': '74004000', 'investmentIncomeNet': 'None', 'netInterestIncome': '-9338000', 'interestIncome': '7832000', 'interestExpense': '17170000', 'nonInterestIncome': '603206000', 'otherNonOperatingIncome': 'None', 'depreciation': '493000', 'depreciationAndAmortization': '183222000', 'incomeBeforeTax': '271908000', 'incomeTaxExpense': '32926000', 'interestAndDebtExpense': 'None', 'netIncomeFromContinuingOperations': '239942000', 'comprehensiveIncomeNetOfTax': '238982000', 'ebit': '283709000', 'ebitda': '474763000', 'netIncome': '238982000'}
first element of the quarterlyReports: {'fiscalDateEnding': '2024-06-30', 'reportedCurrency': 'USD', '

In [3]:
df_income_annual = pl.DataFrame(rgld_income[0]['annualReports'])
df_income_quarterly = pl.DataFrame(rgld_income[0]['quarterlyReports'])

df_income_annual.head()

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2023-12-31""","""USD""","""343394000""","""605717000""","""130284000""","""90523000""","""303202000""","""39761000""","""None""","""79953000""","""None""","""-20915000""","""9952000""","""30867000""","""605717000""","""None""","""431000""","""191054000""","""281448000""","""42008000""","""None""","""240132000""","""239440000""","""303202000""","""468139000""","""239440000"""
"""2022-12-31""","""USD""","""323101000""","""603206000""","""133541000""","""94642000""","""283709000""","""34612000""","""None""","""74004000""","""None""","""-9338000""","""7832000""","""17170000""","""603206000""","""None""","""493000""","""183222000""","""271908000""","""32926000""","""None""","""239942000""","""238982000""","""283709000""","""474763000""","""238982000"""
"""2021-06-30""","""USD""","""333002000""","""648614000""","""121285000""","""92898000""","""337602000""","""28387000""","""None""","""57693000""","""None""","""-1148000""","""None""","""1148000""","""649762000""","""None""","""356000""","""None""","""339399000""","""36867000""","""1148000""","""302776000""","""302532000""","""337602000""","""None""","""302532000"""
"""2020-06-30""","""USD""","""236058000""","""498819000""","""115426000""","""83890000""","""198945000""","""30195000""","""None""","""36726000""","""None""","""-1136000""","""None""","""1136000""","""498819000""","""None""","""387000""","""None""","""195689000""","""-3654000""","""1136000""","""196250000""","""199343000""","""198945000""","""None""","""199343000"""
"""2019-06-30""","""USD""","""178553000""","""423056000""","""108023000""","""77535000""","""140707000""","""30488000""","""None""","""37646000""","""None""","""-15288000""","""None""","""15288000""","""423056000""","""None""","""200000""","""None""","""111323000""","""17498000""","""15288000""","""89079000""","""93825000""","""140707000""","""None""","""93825000"""


In [24]:
df_income_quarterly.head()

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""2024-09-30""","""USD""","""129034000""","""193837000""","""37380000""","""27192000""","""118846000""","""10102000""","""None""","""10188000""","""None""","""-581000""","""626000""","""1207000""","""193837000""","""None""","""None""","""86000""","""117752000""","""21510000""","""None""","""96330000""","""96242000""","""118846000""","""118932000""","""96242000"""
"""2024-06-30""","""USD""","""112683000""","""174096000""","""34774000""","""24174000""","""102083000""","""10511000""","""None""","""10600000""","""None""","""-1709000""","""807000""","""2516000""","""174096000""","""None""","""None""","""89000""","""100199000""","""18991000""","""None""","""81320000""","""81208000""","""102083000""","""102172000""","""81208000"""
"""2024-03-31""","""USD""","""87019000""","""148902000""","""33245000""","""21751000""","""75525000""","""11412000""","""None""","""11494000""","""None""","""-1630000""","""2977000""","""4607000""","""148902000""","""None""","""None""","""82000""","""74199000""","""27033000""","""None""","""47309000""","""47166000""","""75525000""","""75607000""","""47166000"""
"""2023-12-31""","""USD""","""89516000""","""152666000""","""30180000""","""20785000""","""79690000""","""9741000""","""None""","""49587000""","""None""","""-3396000""","""2604000""","""6000000""","""152666000""","""None""","""None""","""38413000""","""76135000""","""13356000""","""None""","""62962000""","""62779000""","""79690000""","""119780000""","""62779000"""
"""2023-09-30""","""USD""","""75606000""","""138617000""","""31395000""","""21351000""","""65562000""","""9927000""","""None""","""10044000""","""None""","""-4849000""","""2436000""","""7285000""","""138617000""","""None""","""None""","""117000""","""60089000""","""10752000""","""None""","""49499000""","""49337000""","""65562000""","""65679000""","""49337000"""


## Income Statement Data Definition

https://documentation.alphavantage.co/FundamentalDataDocs/gaap_documentation.html#IncomeStatement

In [4]:
income_schema = [
    # 'fiscalDateEnding',
    # 'reportedCurrency',
    'grossProfit',
    'totalRevenue',
    'costOfRevenue',
    'costofGoodsAndServicesSold',
    'operatingIncome',
    'sellingGeneralAndAdministrative',
    'researchAndDevelopment',
    'operatingExpenses',
    'investmentIncomeNet',
    'netInterestIncome',
    'interestIncome',
    'interestExpense',
    'nonInterestIncome',
    'otherNonOperatingIncome',
    'depreciation',
    'depreciationAndAmortization',
    'incomeBeforeTax',
    'incomeTaxExpense',
    'interestAndDebtExpense',
    'netIncomeFromContinuingOperations',
    'comprehensiveIncomeNetOfTax',
    'ebit',
    'ebitda',
    'netIncome'
]

for column in income_schema:
    df_income_annual = df_income_annual.with_columns(pl.col(column).cast(pl.Float64, strict=False))
    df_income_quarterly = df_income_quarterly.with_columns(pl.col(column).cast(pl.Float64, strict=False))

df_income_annual = df_income_annual.with_columns(pl.col('fiscalDateEnding').str.to_date('%Y-%m-%d'))
df_income_quarterly = df_income_quarterly.with_columns(pl.col('fiscalDateEnding').str.to_date('%Y-%m-%d'))
df_income_annual.head()

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-12-31,"""USD""",3.43394e8,6.05717e8,1.30284e8,9.0523e7,3.03202e8,3.9761e7,null,7.9953e7,null,-2.0915e7,9.952e6,3.0867e7,6.05717e8,null,431000.0,1.91054e8,2.81448e8,4.2008e7,null,2.40132e8,2.3944e8,3.03202e8,4.68139e8,2.3944e8
2022-12-31,"""USD""",3.23101e8,6.03206e8,1.33541e8,9.4642e7,2.83709e8,3.4612e7,null,7.4004e7,null,-9.338e6,7.832e6,1.717e7,6.03206e8,null,493000.0,1.83222e8,2.71908e8,3.2926e7,null,2.39942e8,2.38982e8,2.83709e8,4.74763e8,2.38982e8
2021-06-30,"""USD""",3.33002e8,6.48614e8,1.21285e8,9.2898e7,3.37602e8,2.8387e7,null,5.7693e7,null,-1.148e6,null,1.148e6,6.49762e8,null,356000.0,null,3.39399e8,3.6867e7,1.148e6,3.02776e8,3.02532e8,3.37602e8,null,3.02532e8
2020-06-30,"""USD""",2.36058e8,4.98819e8,1.15426e8,8.389e7,1.98945e8,3.0195e7,null,3.6726e7,null,-1.136e6,null,1.136e6,4.98819e8,null,387000.0,null,1.95689e8,-3.654e6,1.136e6,1.9625e8,1.99343e8,1.98945e8,null,1.99343e8
2019-06-30,"""USD""",1.78553e8,4.23056e8,1.08023e8,7.7535e7,1.40707e8,3.0488e7,null,3.7646e7,null,-1.5288e7,null,1.5288e7,4.23056e8,null,200000.0,null,1.11323e8,1.7498e7,1.5288e7,8.9079e7,9.3825e7,1.40707e8,null,9.3825e7


In [9]:
BASE_DIR = 'stock_fundamentals'
ticker = 'RGLD'
# load the data to s3
bucket = os.environ["S3_ARB_BUCKET"]
aws_profile = os.environ["S3_PROFILE"]
# create the s3io object 
aws_s3 = S3IO(bucket=bucket, profile=aws_profile)

s3_path = f"{BASE_DIR}/income/annual/{ticker}/data.parq"
aws_s3.s3_write_parquet(df=df_income_annual, file_path=s3_path)

In [11]:
# read the data
df_read = aws_s3.s3_read_parquet(file_path=s3_path)
df_read.head()

fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-12-31,"""USD""",3.43394e8,6.05717e8,1.30284e8,9.0523e7,3.03202e8,3.9761e7,null,7.9953e7,null,-2.0915e7,9.952e6,3.0867e7,6.05717e8,null,431000.0,1.91054e8,2.81448e8,4.2008e7,null,2.40132e8,2.3944e8,3.03202e8,4.68139e8,2.3944e8
2022-12-31,"""USD""",3.23101e8,6.03206e8,1.33541e8,9.4642e7,2.83709e8,3.4612e7,null,7.4004e7,null,-9.338e6,7.832e6,1.717e7,6.03206e8,null,493000.0,1.83222e8,2.71908e8,3.2926e7,null,2.39942e8,2.38982e8,2.83709e8,4.74763e8,2.38982e8
2021-06-30,"""USD""",3.33002e8,6.48614e8,1.21285e8,9.2898e7,3.37602e8,2.8387e7,null,5.7693e7,null,-1.148e6,null,1.148e6,6.49762e8,null,356000.0,null,3.39399e8,3.6867e7,1.148e6,3.02776e8,3.02532e8,3.37602e8,null,3.02532e8
2020-06-30,"""USD""",2.36058e8,4.98819e8,1.15426e8,8.389e7,1.98945e8,3.0195e7,null,3.6726e7,null,-1.136e6,null,1.136e6,4.98819e8,null,387000.0,null,1.95689e8,-3.654e6,1.136e6,1.9625e8,1.99343e8,1.98945e8,null,1.99343e8
2019-06-30,"""USD""",1.78553e8,4.23056e8,1.08023e8,7.7535e7,1.40707e8,3.0488e7,null,3.7646e7,null,-1.5288e7,null,1.5288e7,4.23056e8,null,200000.0,null,1.11323e8,1.7498e7,1.5288e7,8.9079e7,9.3825e7,1.40707e8,null,9.3825e7


# Performing SCD2

AlphaVantage when a request is made loads all the historical data. Because we will be setting up a job that makes calls continuously to retrieve new information we need a way to update old records if there was a change to the data and add the new records to our parquet file. The fundamental data will be considered a SCD2 type file given new data for the quarterly will be on a quarterly basis and the yearly will be on a yearly basis. For this type of data we are not worried about deleting any information. Only what is going to be updated and what will be added. A flag for is current will be added and received time to know when the record was added. This offers a traceback of the records and when they were ran.

In [9]:
# create a sample data frame to show how the SCD2 will work
df_target = pl.DataFrame({
    'fiscalDateEnding': ['2021-12-31', '2020-12-31', '2019-12-31'],
    'grossProfit': [100, 200, 300],
    'totalRevenue': [1000, 2000, 3000],
    'netIncome': [10, 20, 30],
    'is_current': [True, True, True],
    'update_time': [datetime.now(), datetime.now(), datetime.now()]
})
df_target

fiscalDateEnding,grossProfit,totalRevenue,netIncome,is_current,update_time
str,i64,i64,i64,bool,datetime[μs]
"""2021-12-31""",100,1000,10,true,2025-01-12 11:47:34.942200
"""2020-12-31""",200,2000,20,true,2025-01-12 11:47:34.943662
"""2019-12-31""",300,3000,30,true,2025-01-12 11:47:34.943663


In [5]:
# create a new data frame with a record that has been updated and a new record
df_source = pl.DataFrame({
    'fiscalDateEnding': ['2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31'],
    'grossProfit': [200, 200, 300, 400],
    'totalRevenue': [2000, 2000, 4000, 3000],
    'netIncome': [10, 20, 30, 40],
})
df_source

fiscalDateEnding,grossProfit,totalRevenue,netIncome
str,i64,i64,i64
"""2021-12-31""",200,2000,10
"""2020-12-31""",200,2000,20
"""2019-12-31""",300,4000,30
"""2018-12-31""",400,3000,40


In [6]:
# merge the two data frames on the fiscalDateEnding to find the records that need to be updated
updated_records_df = df_sample.filter(
    pl.col('is_current') == True,
).join(
    other=df_new,
    on='fiscalDateEnding',
    how='inner'
)

updated_records_df

fiscalDateEnding,grossProfit,totalRevenue,netIncome,is_current,update_time,grossProfit_right,totalRevenue_right,netIncome_right
str,i64,i64,i64,bool,datetime[μs],i64,i64,i64
"""2021-12-31""",100,1000,10,true,2025-01-05 10:39:15.248800,200,2000,10
"""2020-12-31""",200,2000,20,true,2025-01-05 10:39:15.248858,200,2000,20
"""2019-12-31""",300,3000,30,true,2025-01-05 10:39:15.248859,300,4000,30


In [13]:
# find the records that differ between the two data frames
df_diff = df_new.join(df_sample, on='fiscalDateEnding', suffix='_df2') .filter(pl.any_horizontal(
                                                                            pl.col(x).ne_missing(pl.col(f"{x}_df2"))
                                                                                for x in df_new.columns if x!='fiscalDateEnding'
                                                                            )).select(df_new.columns)
# add the update time and is_current flag to the df_diff
df_diff = df_diff.with_columns(
    pl.lit(True).alias("is_current"),
    pl.lit(datetime.now()).alias("update_time")
)

df_diff

fiscalDateEnding,grossProfit,totalRevenue,netIncome,is_current,update_time
str,i64,i64,i64,bool,datetime[μs]
"""2021-12-31""",200,2000,10,true,2025-01-05 12:12:36.766524
"""2019-12-31""",300,4000,30,true,2025-01-05 12:12:36.766524


In [16]:
# get the list of fiscalDateEnding that are in the df_diff
fiscal_date_list = df_diff.select('fiscalDateEnding').to_series()
# update the is current flag for the old records
df_sample = df_sample.with_columns(
                is_current=pl.when(pl.col('fiscalDateEnding').is_in(fiscal_date_list))
                        .then(pl.lit(False))
                        .otherwise(pl.col('is_current')))
# concat the two data frames and order by fiscalDateEnding
df_sample = pl.concat([df_sample, df_diff]).sort('fiscalDateEnding')
df_sample

fiscalDateEnding,grossProfit,totalRevenue,netIncome,is_current,update_time
str,i64,i64,i64,bool,datetime[μs]
"""2019-12-31""",300,3000,30,false,2025-01-05 10:39:15.248859
"""2019-12-31""",300,4000,30,true,2025-01-05 12:12:36.766524
"""2020-12-31""",200,2000,20,true,2025-01-05 10:39:15.248858
"""2021-12-31""",100,1000,10,false,2025-01-05 10:39:15.248800
"""2021-12-31""",200,2000,10,true,2025-01-05 12:12:36.766524


## Performing updates to the Schema if it changes

Scenerios for testing:

- A new column is introduced
- A column is removed
- A column is renamed

In [8]:
df_source_new_col = pl.DataFrame({
    'fiscalDateEnding': ['2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31'],
    'grossProfit': [200, 200, 300, 400],
    'totalRevenue': [2000, 2000, 4000, 3000],
    'totalExpenses': [100, 200, 300, 400],
    'netIncome': [10, 20, 30, 40],
})

df_source_remove_col = pl.DataFrame({
    'fiscalDateEnding': ['2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31'],
    'grossProfit': [200, 200, 300, 400],
    'netIncome': [10, 20, 30, 40],
})

df_source_rename_col = pl.DataFrame({
    'fiscalDateEnding': ['2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31'],
    'grossProfit': [200, 200, 300, 400],
    'totalRevenues_gold': [2000, 2000, 4000, 3000],
    'netIncome': [10, 20, 30, 40],
})

In [10]:
# get the new columns and merge to the target data frame
new_column = list(set(df_source_new_col.columns) - set(df_target.columns))
new_column.append('fiscalDateEnding')
# join the dataframe on the fiscalDateEnding
df_target_new_col = df_target.join(df_source_new_col.select(new_column), on='fiscalDateEnding', how='left')
df_target_new_col

fiscalDateEnding,grossProfit,totalRevenue,netIncome,is_current,update_time,totalExpenses
str,i64,i64,i64,bool,datetime[μs],i64
"""2021-12-31""",100,1000,10,true,2025-01-12 11:47:34.942200,100
"""2020-12-31""",200,2000,20,true,2025-01-12 11:47:34.943662,200
"""2019-12-31""",300,3000,30,true,2025-01-12 11:47:34.943663,300


In [11]:
# get the removed columns and add it to the source data frame as null
field_cols = [x for x in df_target.columns if x not in ['fiscalDateEnding', 'is_current', 'update_time']]
removed_column = list(set(df_target.select(field_cols).columns) - set(df_source_remove_col.columns))
# join the dataframe on the fiscalDateEnding
df_source_remove_col = df_source_remove_col.with_columns(pl.lit(None).alias(x) for x in removed_column)
df_source_remove_col

fiscalDateEnding,grossProfit,netIncome,totalRevenue
str,i64,i64,null
"""2021-12-31""",200,10,null
"""2020-12-31""",200,20,null
"""2019-12-31""",300,30,null
"""2018-12-31""",400,40,null


In [12]:
# handiling the renamed columns
removed_column = list(set(df_target.select(field_cols).columns) - set(df_source_rename_col.columns))
# join the dataframe on the fiscalDateEnding
df_source_rename_col = df_source_rename_col.with_columns(pl.lit(None).alias(x) for x in removed_column)

new_column = list(set(df_source_rename_col.columns) - set(df_target.columns))
new_column.append('fiscalDateEnding')
# join the dataframe on the fiscalDateEnding
df_target_rename_col = df_target.join(df_source_rename_col.select(new_column), on='fiscalDateEnding', how='left')
print(df_target_rename_col)
print(df_source_rename_col)

shape: (3, 7)
┌──────────────┬─────────────┬──────────────┬───────────┬────────────┬──────────────┬──────────────┐
│ fiscalDateEn ┆ grossProfit ┆ totalRevenue ┆ netIncome ┆ is_current ┆ update_time  ┆ totalRevenue │
│ ding         ┆ ---         ┆ ---          ┆ ---       ┆ ---        ┆ ---          ┆ s_gold       │
│ ---          ┆ i64         ┆ i64          ┆ i64       ┆ bool       ┆ datetime[μs] ┆ ---          │
│ str          ┆             ┆              ┆           ┆            ┆              ┆ i64          │
╞══════════════╪═════════════╪══════════════╪═══════════╪════════════╪══════════════╪══════════════╡
│ 2021-12-31   ┆ 100         ┆ 1000         ┆ 10        ┆ true       ┆ 2025-01-12   ┆ 2000         │
│              ┆             ┆              ┆           ┆            ┆ 11:47:34.942 ┆              │
│              ┆             ┆              ┆           ┆            ┆ 200          ┆              │
│ 2020-12-31   ┆ 200         ┆ 2000         ┆ 20        ┆ true       ┆ 2025-0

In [13]:
# Create an update function that will handle the updates to the data frame using a source and target data frame
def update_records(
        target: pl.DataFrame,
        source: pl.DataFrame,
        on: str='fiscalDateEnding') -> pl.DataFrame:
    """
    The following function looks to update the records of a slowly changing dimension type 2 data frame. Using a source dataframe
    
    Parameters:
    target (pl.DataFrame): The target data frame that will be updated
    source (pl.DataFrame): The source data frame that will be used to update the target data frame
    
    
    Returns:
    pl.DataFrame: The updated target data frame
    """
    # find the records that differ between the two data frames
    diff = source.join(target, on=on, suffix='_df2') .filter(pl.any_horizontal(
                                                                            pl.col(x).ne_missing(pl.col(f"{x}_df2"))
                                                                                for x in source.columns if x!='fiscalDateEnding')).select(source.columns)
    # add the update time and is_current flag to the df_diff
    diff = diff.with_columns(
        pl.lit(True).alias("is_current"),
        pl.lit(datetime.now()).alias("update_time")
    )
    # get the list of fiscalDateEnding that are in the df_diff
    date_list = diff.select('fiscalDateEnding').to_series()
    # update the is current flag for the old records
    df_updated = target.with_columns(
                    is_current=pl.when(pl.col('fiscalDateEnding').is_in(date_list))
                            .then(pl.lit(False))
                            .otherwise(pl.col('is_current')))
    # concat the two data frames and order by fiscalDateEnding
    print(f"Shape of target: {df_updated.shape}")
    print(f"Shape of diff: {diff.shape}")
    df_updated = pl.concat([df_updated, diff.select(df_updated.columns)]).sort('fiscalDateEnding')
    return df_updated


In [14]:
# test the update function on th 3 dataframes
df_new_col = update_records(target=df_target_new_col, source=df_source_new_col)
df_remove_col = update_records(target=df_target, source=df_source_remove_col)
df_rename_col = update_records(target=df_target_rename_col, source=df_source_rename_col)

print(df_new_col)
print(df_remove_col)
print(df_rename_col)

Shape of target: (3, 7)
Shape of diff: (2, 7)
Shape of target: (3, 6)
Shape of diff: (3, 6)
Shape of target: (3, 7)
Shape of diff: (3, 7)
shape: (5, 7)
┌──────────────┬─────────────┬──────────────┬───────────┬────────────┬──────────────┬──────────────┐
│ fiscalDateEn ┆ grossProfit ┆ totalRevenue ┆ netIncome ┆ is_current ┆ update_time  ┆ totalExpense │
│ ding         ┆ ---         ┆ ---          ┆ ---       ┆ ---        ┆ ---          ┆ s            │
│ ---          ┆ i64         ┆ i64          ┆ i64       ┆ bool       ┆ datetime[μs] ┆ ---          │
│ str          ┆             ┆              ┆           ┆            ┆              ┆ i64          │
╞══════════════╪═════════════╪══════════════╪═══════════╪════════════╪══════════════╪══════════════╡
│ 2019-12-31   ┆ 300         ┆ 3000         ┆ 30        ┆ false      ┆ 2025-01-12   ┆ 300          │
│              ┆             ┆              ┆           ┆            ┆ 11:47:34.943 ┆              │
│              ┆             ┆          

In [15]:
print(df_target_new_col)
print(df_source_new_col)

shape: (3, 7)
┌──────────────┬─────────────┬──────────────┬───────────┬────────────┬──────────────┬──────────────┐
│ fiscalDateEn ┆ grossProfit ┆ totalRevenue ┆ netIncome ┆ is_current ┆ update_time  ┆ totalExpense │
│ ding         ┆ ---         ┆ ---          ┆ ---       ┆ ---        ┆ ---          ┆ s            │
│ ---          ┆ i64         ┆ i64          ┆ i64       ┆ bool       ┆ datetime[μs] ┆ ---          │
│ str          ┆             ┆              ┆           ┆            ┆              ┆ i64          │
╞══════════════╪═════════════╪══════════════╪═══════════╪════════════╪══════════════╪══════════════╡
│ 2021-12-31   ┆ 100         ┆ 1000         ┆ 10        ┆ true       ┆ 2025-01-12   ┆ 100          │
│              ┆             ┆              ┆           ┆            ┆ 11:47:34.942 ┆              │
│              ┆             ┆              ┆           ┆            ┆ 200          ┆              │
│ 2020-12-31   ┆ 200         ┆ 2000         ┆ 20        ┆ true       ┆ 2025-0

shape: (6, 7)
┌──────────────┬─────────────┬──────────────┬───────────┬────────────┬──────────────┬──────────────┐
│ fiscalDateEn ┆ grossProfit ┆ totalRevenue ┆ netIncome ┆ is_current ┆ update_time  ┆ totalExpense │
│ ding         ┆ ---         ┆ ---          ┆ ---       ┆ ---        ┆ ---          ┆ s            │
│ ---          ┆ i64         ┆ i64          ┆ i64       ┆ bool       ┆ datetime[μs] ┆ ---          │
│ str          ┆             ┆              ┆           ┆            ┆              ┆ i64          │
╞══════════════╪═════════════╪══════════════╪═══════════╪════════════╪══════════════╪══════════════╡
│ 2018-12-31   ┆ 400         ┆ 3000         ┆ 40        ┆ true       ┆ 2025-01-12   ┆ 400          │
│              ┆             ┆              ┆           ┆            ┆ 11:51:28.758 ┆              │
│              ┆             ┆              ┆           ┆            ┆ 134          ┆              │
│ 2019-12-31   ┆ 300         ┆ 3000         ┆ 30        ┆ false      ┆ 2025-0